# Download dei dati 
Supponiamo di non avere i dati, quindi dobbiamo scaricarli tutti.

Quello che viene fatto è fare scraping attraverso https://fbref.com/it/comp/11/2021-2022/Statistiche-di-Serie-A-2021-2022 nella stagione 2021-2022 e vengono estratte tutte le informazioni relative alle varie squadre che giocano nella stagione. 

I dati provenienti da diversi link vengono mergiati. 
Per poter classificare ho pensato a 3 classi: V (vittoria), P (persa), N (pareggio).
Il problema è che scaricando tutte le partite per tutte le squadre, ci saranno partite doppie, ad esempio: 
- Atalanta V Bologna
- Bologna P Atalanta

Per questo vado a considerare nella classificazione solo le squadre in casa, così quando si vuole predire il risultato di una partita, ottengo la classificazione per la squadra di casa.


# Dataset Serie A matches
Per la realizzazione del dataset ho trovato in letteratura l'elenco dei dati più importanti per la predizione dei risultati delle partite. 

https://iopscience.iop.org/article/10.1088/1757-899X/226/1/012099/pdf

Per questo dataset ho utilizzato le statistiche della stagione 2021/22

## Scaricamento di tutti i match
In questa sezione vengono scaricate tutte le statistiche delle partite giocate da ogni singola squadra nel campionato.
Le statistiche in un match sono relative alla squadra presa in considerazione, non all'avversario (indipendentemente da che sia in casa o in trasferta).

In [1]:
from download import DownloadDati
import pandas as pd
import time
import util_strings as utils

Un modello precisio, deve avere tante partite e in questo caso, più vengono considerati match, meglio è per il modello.

Nel codice sotto viene scaricata la stagione attuale

In [2]:
#dizionario contenente il nome della stagione (anno) come chiave e come valore il link in cui viene presa la stagione

#Per la premier league: https://fbref.com/it/comp/9/Statistiche-di-Premier-League
download_seasons = {
    "2022-2023": "https://fbref.com/it/comp/11/Statistiche-di-Serie-A",
    #"2021-2022": "https://fbref.com/it/comp/11/2021-2022/Statistiche-di-Serie-A-2021-2022",
    #"2020-2021": "https://fbref.com/it/comp/11/2020-2021/Statistiche-di-Serie-A-2020-2021",
    #"2019-2020": "https://fbref.com/it/comp/11/2019-2020/Statistiche-di-Serie-A-2019-2020",
    #"2018-2019": "https://fbref.com/it/comp/11/2018-2019/Statistiche-di-Serie-A-2018-2019"
}

In [5]:
downloadDF = pd.DataFrame()
row = 0
for season, link in download_seasons.items():
    print("Season "+season+ " downloading...")
    start = time.time()
    
    download = DownloadDati("Serie A") 
    download.connect(link)
    download.get_teams_names()
    download.get_matches()
    download.save_matches(utils.statistics.format(season))
    download.save_championship_games(utils.championship.format(season))

    print("Season "+season+ " download ended\n")
    end = time.time()
    downloadDF.at[row, 'season'] = season
    downloadDF.at[row, 'download time'] = end-start
    downloadDF.at[row, 'n. matches'] = len(download.all_matches)/2 
    row += 1
    

Season 2022-2023 downloading...


ValueError: No tables found

## Merge delle partite
Leggo tutti i file presenti nella cartella ed effettuo il merge

In [ ]:
from os import listdir, statvfs
from os.path import isfile, join
import pandas as pd
from analysis import MatchAnalysis

onlyfiles = [f for f in listdir(utils.stats) if isfile(join(utils.stats, f))]
name_csv_statistics_FE_season = [x for x in onlyfiles] 

merged_statistics = pd.DataFrame()

for name_statistics_single_season in sorted(name_csv_statistics_FE_season):
    print(utils.stats+name_statistics_single_season)
    statistics_single_season = pd.read_csv(utils.stats+name_statistics_single_season, index_col=0)

    #setto il nome della stagione come i 9 primi caratteri del nome del file
    statistics_single_season['season'] = name_statistics_single_season[:9] #es: 2022-2023
    merged_statistics = merged_statistics.append(statistics_single_season)

Serie A/Stats/2018-2019.csv
Serie A/Stats/2019-2020.csv
Serie A/Stats/2020-2021.csv
Serie A/Stats/2021-2022.csv
Serie A/Stats/2022-2023.csv


In [ ]:
merged_statistics['team1'] = merged_statistics['team1'].str.lower()
merged_statistics['team2'] = merged_statistics['team2'].str.lower()

In [ ]:
merged_statistics.to_csv(utils.merged_statistics)